In [ ]:
import numpy as np
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import spsolve

In [ ]:
# Coeficiente de difusão q(u) = 1 + u^2
def q_func(u):
    return 1.0 + u**2

# Vetor da solução exata u(x,y) = sin(πx)sin(πy)
def calculate_exact_solution(N, h):
    
    exact_u = np.zeros((N, N))
    
    for i in range(N):
        for j in range(N):
            x = (i + 1) * h
            y = (j + 1) * h
            exact_u[i, j] = np.sin(np.pi * x) * np.sin(np.pi * y)
    
    return exact_u.flatten()

# Compara a solução numérica com a exata
def calculate_error(numerical_U, exact_U, h):
    
    error_vector = numerical_U - exact_U
    l2_error = np.sqrt(h**2 * np.sum(error_vector**2))
    max_error = np.max(np.abs(error_vector))
    
    return l2_error, max_error

# Cálculo do vetor do sistema não linear F(U)
def calculate_F(U_vector, N, h, source_f):
    
    u = np.zeros((N + 2, N + 2))
    u[1:-1, 1:-1] = U_vector.reshape((N, N))
    F = np.zeros((N, N))

    for i in range(N):
        for j in range(N):
            r, c = i + 1, j + 1
            
            u_c = u[r, c]
            u_n = u[r+1,c]
            u_s = u[r-1,c]
            u_e = u[r,c+1]
            u_w = u[r,c-1]
            
            q_c = q_func(u_c)
            q_n = q_func(u_n)
            q_s = q_func(u_s)
            q_e = q_func(u_e)
            q_w = q_func(u_w)

            q_ip12j = (q_e + q_c) / 2.0
            q_im12j = (q_w + q_c) / 2.0
            q_ijp12 = (q_n + q_c) / 2.0
            q_ijm12 = (q_s + q_c) / 2.0

            x_term = q_ip12j * (u_e - u_c) - q_im12j * (u_c - u_w)
            y_term = q_ijp12 * (u_n - u_c) - q_ijm12 * (u_c - u_s)
            
            f_ij = source_f[i * N + j]
            F[i, j] = (x_term + y_term) / h**2 - f_ij
            
    return F.flatten()

# Monta a matriz Jacobiana J = dF/dU no formato CSR
def jacobian_csr(U_vector, N, h):
    
    u = np.zeros((N + 2, N + 2))
    u[1:-1, 1:-1] = U_vector.reshape((N, N))
    
    values = []
    column_indices = []
    row_pointers = [0]

    for i in range(N):
        for j in range(N):
            r, c = i + 1, j + 1
            
            u_c = u[r,c]
            u_n = u[r+1,c]
            u_s = u[r-1,c]
            u_e = u[r,c+1]
            u_w = u[r,c-1]
            
            q_c = q_func(u_c)
            q_n = q_func(u_n)
            q_s = q_func(u_s)
            q_e = q_func(u_e)
            q_w = q_func(u_w)

            # dF/du_{i,j-1} (Oeste)
            if j > 0:
                val = (q_w + q_c)/2.0 - u_w * (u_c - u_w) 
                column_indices.append(i * N + (j - 1))
                values.append(val / h**2)

            # dF/du_{i-1,j} (Sul)
            if i > 0:
                val = (q_s + q_c)/2.0 - u_s * (u_c - u_s) 
                column_indices.append((i - 1) * N + j)
                values.append(val / h**2)
            
            # dF/du_{i,j} (Central)
            val_c = (-(q_e+q_w+q_n+q_s)/2.0 - 2*q_c) - u_c * ((u_e-u_c) + (u_w-u_c) + (u_n-u_c) + (u_s-u_c)) 
            column_indices.append(i * N + j)
            values.append(val_c / h**2)

            # dF/du_{i+1,j} (Norte)
            if i < N - 1:
                val = (q_n + q_c)/2.0 - u_n * (u_c - u_n) 
                column_indices.append((i + 1) * N + j)
                values.append(val / h**2)

            # dF/du_{i,j+1} (Leste)
            if j < N - 1:
                val = (q_e + q_c)/2.0 - u_e * (u_c - u_e) 
                column_indices.append(i * N + (j + 1))
                values.append(val / h**2)
            
            row_pointers.append(len(values))
            
    return np.array(values), np.array(column_indices), np.array(row_pointers)

# Resolve o sistema F(U)=0 com o Método de Newton
def solve_newton(initial_U, N, h, source_f, tol=1e-6, max_iter=50):
    
    U_k = np.copy(initial_U) 
    
    print("Iniciando Método de Newton...")
    
    for k in range(max_iter):
        
        F_k = calculate_F(U_k, N, h, source_f)
        J_values, J_cols, J_pointers = jacobian_csr(U_k, N, h)
        
        # s_k = solve_lu_csr(J_values, J_cols, J_pointers, -F_k)
        J_csc = csc_matrix((J_values, J_cols, J_pointers), shape=(N*N, N*N))
        s_k = spsolve(J_csc, -F_k) 

        U_k = U_k + s_k
        
        step_norm = np.linalg.norm(s_k, np.inf)
        print(f"Iteração {k+1:2d}: Norma do passo = {step_norm:.2e}")
        
        if step_norm < tol:
            print(f"\nConvergência atingida em {k+1} iterações.")
            return U_k

    print("\nMáximo de iterações excedido.")
    
    return U_k